In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/Thesis/long_text_with_pos_text.csv', low_memory = False)
text_long = df['bigram_text'].tolist()
print(len(text_long))

Mounted at /content/drive
2999909


In [ ]:
import pandas as pd
import spacy
import re
from joblib import Parallel, delayed
from tqdm import tqdm


def process_tweet(text):
    nlp = spacy.load("en_core_web_sm")

    uncivil_words = {"dumb", "farc", "hypocrit", "insan", "insecur", "irrespons", "sham", "trivial", "unaccept", "uneth", "unfair",
                 "bigot", "bigotri", "discrimin", "prejudic", "racism", "racist", "segreg", "stereotyp", "betray", "enemi",
                 "insurg", "overthrow", "riot", "threat", "threaten", "traitor", "treason", "tyranni", "brainwash", "deceiv",
                 "decept", "dishonest", "disingenu", "inaccur", "incorrect", "misinform", "mislead", "uninform", "dysfunct",
                 "infring", "obstruct", "obstructionist", "suppress", "unconstitut", "chao", "debacl", "delud", "demean",
                 "disrespect", "fiasco", "inappropri", "nasti", "vitriol", "yell"}


    doc = nlp(text)
    lemmas = [token.lemma_.lower() for token in doc if not token.is_space]
    if not lemmas:
        return 0.0
    count = sum(1 for lemma in lemmas if any(lemma.startswith(w) for w in uncivil_words))
    return round(count / len(lemmas), 4)


results = Parallel(n_jobs=42)(
    delayed(process_tweet)(text) for text in tqdm(text_long, desc="Lemmatizing + Scoring")
)

df["uncivil_score"] = results
df.to_csv("/content/drive/MyDrive/Thesis/long_text_with_pos_text.csv", index=False)

Lemmatizing + Scoring: 100%|██████████| 2999909/2999909 [9:05:45<00:00, 91.61it/s]


In [ ]:
df_long = pd.read_csv("/content/drive/MyDrive/Thesis/long_text_with_pos_text.csv", low_memory = False)
print(df_long.head())

                                                text  viewCount  likeCount  \
0  @CynCity1243 @MikeRoy_Cit12 @GastricpH Likewis...       49.0        0.0   
1  @Megatron_ron France should take in Lebanese r...       84.0        1.0   
2  @TiltingNBA @majornija @nicole_naiya Heard thi...        4.0        0.0   
3  Guys.\n\nDonald Trump is quite literally the l...      159.0        6.0   
4  @realDonaldTrump The American people have endu...     2274.0       74.0   

   quoteCount  replyCount  retweetCount  char_count  \
0         0.0         1.0           0.0         320   
1         0.0         0.0           0.0         283   
2         0.0         0.0           0.0         338   
3         0.0         0.0           3.0         303   
4         3.0         3.0          24.0         318   

                                         bigram_text  uncivil_score  
0  likewise kennedy is famously a democrat i was ...         0.0000  
1  france should take in lebanese refugees as wel...        